# Map

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores_colombianos_4.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval,
        'GEOLOC_CIUDAD_AUTOR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 224 ms, sys: 4.02 ms, total: 228 ms
Wall time: 227 ms


In [3]:
autores_df.shape

(3369, 14)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO,NOMBRE_ENCONTRADO,GEOLOC_CIUDAD_AUTOR
0,En la laguna más profunda,Colombia,Oscar Collazos,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bahía Solano,Chocó,True,Oscar Collazos,"(6.222551999999999, -77.40115399999999)"
1,Proyecto para un arca en medio de un diluvio d...,Colombia,Luis Carlos Restrepo,Colombia,1997,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Filandia,Quindío,True,Luis Carlos Restrepo,"(4.675762, -75.659359)"
2,"Octubre, 1942-1952",Colombia,Carmelina Soto Valencia,Colombia,1953,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Armenia,Quindío,True,Carmelina Soto Valencia,"(4.535000399999999, -75.6756888)"
3,Origen y desarrollo del camino del quindio,Colombia,Jesús Arango Cano,Colombia,2002,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",La Tebaida,Quindío,True,Jesús Arango Cano,"(4.453588, -75.79153)"
4,Mi gran aventura cósmica,Colombia,Jesús Arango Cano,Colombia,1976,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",La Tebaida,Quindío,True,Jesús Arango Cano,"(4.453588, -75.79153)"


## Graph

### Rutas

In [5]:
rutas_df = autores_df[autores_df['PAIS_EJEMPLAR'] != 'Colombia']

In [6]:
len(rutas_df)

489

In [7]:
rutas_graph = nx.Graph()

for i, x in rutas_df.iterrows():
    ciudad = x['CIUDAD_AUTOR']
    if not rutas_graph.has_node(ciudad):
        departamento = x['DEPARTAMENTO_AUTOR']
        geoloc = x['GEOLOC_CIUDAD_AUTOR']
        rutas_graph.add_node(
            ciudad,
            departamento=departamento,
            latitude=geoloc[0],
            longitude=geoloc[1],
            weight=1
        )
    else:
        rutas_graph.node[ciudad]['weight'] += 1

In [12]:
rutas_graph.number_of_nodes()

61

### Ciclos

In [8]:
ciclos_df = autores_df[autores_df['PAIS_EJEMPLAR'] == 'Colombia']

In [9]:
len(ciclos_df)

2880

In [10]:
ciclos_graph = nx.Graph()

for i, x in ciclos_df.iterrows():
    ciudad = x['CIUDAD_AUTOR']
    if not ciclos_graph.has_node(ciudad):
        departamento = x['DEPARTAMENTO_AUTOR']
        geoloc = x['GEOLOC_CIUDAD_AUTOR']
        ciclos_graph.add_node(
            ciudad,
            departamento=departamento,
            latitude=geoloc[0],
            longitude=geoloc[1],
            weight=1
        )
    else:
        ciclos_graph.node[ciudad]['weight'] += 1

In [13]:
ciclos_graph.number_of_nodes()

138

# Saving results

In [11]:
nx.write_gexf(ciclos_graph, 'colombianos_ciclos_graph.gexf')
nx.write_gexf(rutas_graph, 'colombianos_rutas_graph.gexf')